# How-To Deploy a Web Application

```{note}
Fork this repository in to your own GitHub account to follow along. GitHub provides detailed documentation on different methods of forking that can be found via this [link to GitHub docs](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/working-with-forks/fork-a-repo?tool=webui).
``` 

## Run the application locally

Let's start by running the application locally. Python and Flask are required. If you do not already have python installed on your machine you can do so by following the instructions found at this [link to Python Downloads](https://www.python.org/downloads/).

With python installed running the following will install Flask

```{note}
The ! is used so the Jupyter notebook cells can be run directly. This is not required if you want to run the code on the command line yourself. 
```

In [ ]:
!pip install -r requirements.txt

Flask is now installed and the application can be launched with the following command

In [ ]:
%cd app
!flask run

The link above should direct you to a local website. If not you can use the following link in your browser : [http://127.0.0.1:5000](http://127.0.0.1:5000)

Use `CTRL+C` to exit the application or stop the running cell if run directly in the jupyter notebook. 

## Create a Container File

```{note}
Docker or Podman are required on your local machine for the next steps. Installation instructions for your OS can be found at this [link to Podman Installation Instructions](https://podman.io/docs/installation) or this [link to Docker Installation Instructions](https://docs.docker.com/engine/install/). If you do not want to build the image locally, using GitHub Actions to build an image is covered below. 

With either Docker or Podman builds the typical set of instructions on how to build a container is called a Dockerfile. 

```
# Use an official Python runtime as a base image
FROM python:3.8-slim

# Copy all the files in the app directory to the root directory in the container
COPY app/. /

RUN pip install --no-cache-dir -r requirements.txt

EXPOSE 5000

CMD ["python3", "app.py"]
```